# Análisis de Métodos

Integrantes: Erick Bey,
Matias Orozco,
Alvaro Parker

# KNN

**Modificaciones posibles:**
* No aplicarle logaritmo a la columna `date_departure_time` (eso implica tener numeros enteros muy grandes).
* Eliminar las columnas `origin` y `destinaton` (no se puede medir distancia con ellas)
* Cambiar el argumento `n_neighbors`a un numero distinto cuando se llama a la funcion `KNeighborsClassifier()`

In [63]:
import numpy as np
import pandas as pd
from plotnine import *
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

def procesar_csv(df):
    from sklearn import preprocessing
    # convertirmos las columnas origin y destination a variables numericas z
    le = preprocessing.LabelEncoder()
    ciudades = ((pd.concat([df['origin'], df['destination']], ignore_index=True).drop_duplicates()).reset_index())[0]
    le.fit(ciudades)
    origin_as_int = le.transform(df['origin'])
    destination_as_int = le.transform(df["destination"])
    
    #gdf["origin_int"] = origin_as_int
    df.drop('origin', axis=1, inplace=True)
    
    #df["destination_int"] = destination_as_int
    df.drop('destination', axis=1, inplace=True)
    df.drop("Unnamed: 0", axis=1, inplace=True)
    # Cambiamos el formato del tiempo a unix time stamp para medir la distancia de forma mas facil.
    df["date_departure_int"] = (pd.to_datetime(df["date"].map(str)+" "+df["departure_time"]).apply(lambda x: x.value))
    #df["date_departure_int"] = np.log10(pd.to_datetime(df["date"].map(str)+" "+df["departure_time"]).apply(lambda x: x.value))
    df.drop('departure_time', axis=1, inplace=True)
    df.drop('date', axis=1, inplace=True)
    # retornamos el data frame modificado sin valore nulos.
    return df.dropna()

In [64]:
# Leemos los datos
df = pd.read_csv("ALUMNOS-trainData.csv")
# Procesamos los datos con la funcion procesar_csv()
df = procesar_csv(df)
# transformamos los datos a binarios
df["nowshow"] = np.where(df["noshow"] >= 4, 1,0)
# Definimos nuestra columna target
target = df['nowshow']
# La eliminamos del dataframe
df.drop('noshow', axis=1, inplace=True)

In [65]:
df

,fligth_number,distance,denied_boarding,pax_midlow,pax_high,pax_midhigh,pax_low,pax_freqflyer,group_bookings,out_of_stock,dom_cnx,int_cnx,p2p,capacity,revenues_usd,bookings,origin_int,destination_int,date_departure_int,nowshow
0,8995,1394,0,94,2,28,94,22,19,0,1,0,239,174.0,10631.1,240,12,125,1257762600000000000,1
1,7061,489,0,100,4,4,87,25,0,0,19,9,192,174.0,5453.7,220,81,143,1243106400000000000,0
2,8972,2180,0,7,1,5,64,2,0,0,0,10,69,218.0,16161.0,79,125,116,1304934840000000000,1
3,6039,524,0,54,4,7,21,4,0,0,26,0,64,174.0,10828.7,90,27,33,1266008400000000000,1
4,7713,1170,0,21,88,18,82,0,88,1,23,3,183,144.0,16721.6,209,53,55,1256388000000000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999885,6024,552,0,25,12,20,4,12,0,0,64,9,0,144.0,14102.1,73,147,27,1255706040000000000,1
999886,6669,3934,0,129,3,19,121,2,20,0,0,118,156,168.0,32907.8,274,81,57,1241340600000000000,1
999887,9094,1430,0,2,1,0,108,20,36,0,7,0,124,218.0,13519.3,131,64,125,1301926680000000000,1
999888,7704,1091,0,67,2,7,42,16,0,0,34,2,98,144.0,8729.6,134,55,126,1238832300000000000,0


In [68]:
from sklearn.model_selection import train_test_split
# Dividimos los data frames en test y train set
X_train, X_test, y_train, y_test = train_test_split(df,target, test_size=0.33,random_state=42)

type(X_train)
type(y_train)

pandas.core.series.Series

### Encontrar los argumentos optimos usando GridSearchCV() 

In [5]:
# Esto va a tomar demasiado, por lo menos 1 dia xd
k_range = (list(range(1001,101001, 1000)))
weight_options = ["uniform", "distance"]

param_grid = dict(n_neighbors = k_range, weights = weight_options)


knn = KNeighborsClassifier()

grid = GridSearchCV(knn, param_grid, cv = 10, scoring = 'accuracy')
grid.fit(X_train,y_train)

print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

### Implementar el KNN con los argumentos encontrados:

In [69]:
#Creando un objeto KNN con las condiciones iniciales
from sklearn.neighbors import KNeighborsClassifier
# Crear el objeto:
KNN = KNeighborsClassifier(n_neighbors=1001,weights="uniform")

#Entrenando el modelo con los datos X_train
KNN=KNN.fit(X_train,y_train)

In [70]:
#Aplicando el modelo a otros datos

# esta cell es la que mas demora 

# Reservamos memoria para la lista q se creara al usar KNN.predict()
# si no hacemos esto, el uso de memoria ram aumenta considerablemente
resultado = pd.Series(np.array([None]*len(y_test)))
# predecimos usando nuestro X_test 
resultado=KNN.predict(X_test)

In [71]:
from sklearn import metrics as mt
# Evaluacion:
accuracy = mt.accuracy_score(y_test, resultado)
precicion_para_cero = mt.precision_score(y_test==0,resultado==0,pos_label=1,average="binary")
precicion_para_uno = mt.precision_score(y_test == 1, resultado == 1, pos_label=1, average="binary")
recall_score = mt.recall_score(y_test, resultado, labels=None, pos_label=1, average = 'binary')
f1_score = mt.f1_score(y_test, resultado, labels=None, pos_label=1, average = 'binary')

In [72]:
print(f"Accuracy: {accuracy}")
print(f"Precicion para 0: {precicion_para_cero}")
print(f"Precicion para 1: {precicion_para_uno}")
print(f"Recall score: {recall_score}")

print(f"f1 Score: {f1_score}")

Accuracy: 0.6609236159096143
Precicion para 0: 0.6481994459833795
Precicion para 1: 0.661208264135042
Recall score: 0.9882375278432535
f1 Score: 0.7923034797701811


In [21]:
print(f"Accuracy: {accuracy}")
print(f"Precicion para 0: {precicion_para_cero}")
print(f"Precicion para 1: {precicion_para_uno}")
print(f"Recall score: {recall_score}")
print(f"f1 Score: {f1_score}")

Accuracy: 0.660086920630296
Precicion para 0: 0.6631508078994613
Precicion para 1: 0.6600449175486094
Recall score: 0.9930522768720896
f1 Score: 0.7930073445429714


### Loops para probar distintos k neigbors

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics as mt


lista_resultados = []

for i in range(1001,101001, 1000):
    iteracion = (int((i-1)/1000))
    print("Iteracion: ", iteracion)
    KNN = KNeighborsClassifier(n_neighbors=i,weights="uniform")
    # Entrenando el modelo con los datos X_train
    KNN=KNN.fit(X_train,y_train)

    # predecimos usando nuestro X_test 
    print("Prediciendo")
    resultado=KNN.predict(X_test)

    lista_resultados.append(resultado)

Iteracion:  1
Prediciendo
Iteracion:  2
Prediciendo
